In [51]:
%run /home/dbaciur/NTU/NTU/notes/Constants.ipynb

Constants loaded


In this file we want to make a first processing of all annotation to remove unwanted columns, fix video file names and remove annotations whose videos doesn't exists.

In [47]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
import torchvision as tv
from datetime import datetime, timedelta

## Import annotations

In [5]:
annotations_df = pd.read_csv(C.ORIGINAL_ANNOTATIONS_FILE, sep='\t')

/home/bjastrze/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
annotations_df.head(1)

,Unnamed: 0,Behavior,Behavior type,Behavioral category,Comment start,Comment stop,Duration (s),FPS,Media file,Modifiers,Observation date,Observation id,Start (s),Stop (s),Subject,Total length,lecture_id,research_assistant_id,subject_id
0,0,eye contact,STATE,Encouraging learner engagement,NaN,NaN,2841.008,29.83,C:/Users/ACER/Downloads/PH1011-PHYSICS_2015092...,NaN,2019-11-30 08:32:25,J1S1C11L1-I10-RA13,29.355,2870.363,No focal subject,7197.58,J1S1C11L1,RA13,J1S1C11


### Assign ID to each event

In [7]:
annotations_df = annotations_df.reset_index().rename(columns={'index':C.A_EVENT_ID})
annotations_df.head(1)

,Id,Unnamed: 0,Behavior,Behavior type,Behavioral category,Comment start,Comment stop,Duration (s),FPS,Media file,Modifiers,Observation date,Observation id,Start (s),Stop (s),Subject,Total length,lecture_id,research_assistant_id,subject_id
0,0,0,eye contact,STATE,Encouraging learner engagement,NaN,NaN,2841.008,29.83,C:/Users/ACER/Downloads/PH1011-PHYSICS_2015092...,NaN,2019-11-30 08:32:25,J1S1C11L1-I10-RA13,29.355,2870.363,No focal subject,7197.58,J1S1C11L1,RA13,J1S1C11


### Assign ID to each behavior

In [8]:
annotations_df[C.A_BEH_ID] = annotations_df[C.A_BEHAVIOR].apply(lambda beh: C.BEH_IDS[beh])
annotations_df.head(1)

,Id,Unnamed: 0,Behavior,Behavior type,Behavioral category,Comment start,Comment stop,Duration (s),FPS,Media file,...,Observation date,Observation id,Start (s),Stop (s),Subject,Total length,lecture_id,research_assistant_id,subject_id,BehaviorId
0,0,0,eye contact,STATE,Encouraging learner engagement,NaN,NaN,2841.008,29.83,C:/Users/ACER/Downloads/PH1011-PHYSICS_2015092...,...,2019-11-30 08:32:25,J1S1C11L1-I10-RA13,29.355,2870.363,No focal subject,7197.58,J1S1C11L1,RA13,J1S1C11,B_EYE


### Set video name

Remove directory path and file extension

In [9]:
annotations_df[C.A_VIDEO] = annotations_df[C.A_MEDIA_FILE].apply(lambda mediafile: os.path.basename(mediafile)[:-4])
annotations_df.head(1)

,Id,Unnamed: 0,Behavior,Behavior type,Behavioral category,Comment start,Comment stop,Duration (s),FPS,Media file,...,Observation id,Start (s),Stop (s),Subject,Total length,lecture_id,research_assistant_id,subject_id,BehaviorId,Video
0,0,0,eye contact,STATE,Encouraging learner engagement,NaN,NaN,2841.008,29.83,C:/Users/ACER/Downloads/PH1011-PHYSICS_2015092...,...,J1S1C11L1-I10-RA13,29.355,2870.363,No focal subject,7197.58,J1S1C11L1,RA13,J1S1C11,B_EYE,PH1011-PHYSICS_20150922


### Check if all videos exists

In [10]:
set_of_all_videos = annotations_df[C.A_VIDEO].unique()

In [11]:
video_exists_dict = {}
for video in set_of_all_videos:
    video_path = os.path.join(C.ORIGINAL_VIDEOS_DIR, video + ".mp4")
    video_exists = os.path.exists(video_path)
    video_exists_dict[video] = video_exists
video_exists_dict

{'PH1011-PHYSICS_20150922': True,
 'PH1011-PHYSICS_20151006': True,
 'PH1011-PHYSICS_20151013': True,
 'PH1011-PHYSICS_20151020': True,
 '15S1-PH1012-LEC_20150812': True,
 'PH1104-MECHANICS_20150817_PC1': True,
 '15S1-PH1104-LEC_20150820_PC1': True,
 'PH1104-MECHANICS_20150824_PC1': True,
 'PH1104-MECHANICS_20150827_PC1': True,
 'PH1104-MECHANICS_20150831_PC1': True,
 '20150817': False,
 '20150921': False,
 '20150821': False,
 '20150824': False,
 '20150828': False,
 '20150828.boris': False,
 '20150831': False,
 '20150904': False,
 '20150907': False,
 '20150914': False,
 '20150918': False,
 '15S1-MH1800-LEC_20150817': True,
 '15S1-MH1800-LEC_20150824': True,
 '15S1-MH1800-LEC_20150831': True,
 '15S2-PH1106-LEC_20160113': True,
 '15S2-PH1106-LEC_20160120': True,
 's3': False,
 '15S2-PH1106-LEC_20160127': True,
 's4': False,
 '15S2-PH1106-LEC_20160203': True,
 's5': False,
 '15S2-PH1106-LEC_20160210': True,
 '15S2-MH1402-LEC_20160115': True,
 '15S2-MH1402-LEC_20160122': True,
 '15S2-MH140

**Result**: definately not all videos exist. We need to fix that.

## Read file with coders annotation-video mappings

This additional file can be treated as some sort of errata. It contains alternative video file names - maybe they do exist and can replace old not existing ones

In [12]:
coders_annotations_df = pd.read_csv(C.ERRATA_ANNOTATIONS_FILE, sep=";")
coders_annotations_df.head(1)

,Semester (year when the lecture toook place + semester no.),Semester annonymisation,Title,Start Date,Start Time,End Time,Studio ID (lecture theatre),Author ID,School,Blackboard Course ID,Course ID,Course ID annonymisation,Status,Lecture duration,Approximate lecture duration,INSTRUCTOR'S ID,Partial LECTURE'S ID,Lecture no.,Partial lecture ID without RA no.,Old Obs. Name
0,17S1,J3S1,17S1-CM1021-LEC_20170823,23.08.2017,12:30:00,13:26:00,LT23-PC1,sumod,SPMS,17S1-CM1021-LEC,CM1021-LEC,C1,Published,00:56:00,1,I12,J3S1C1,L1,J3S1C1L1-I12,a3


### Add lecture ID to coders

In [13]:
coders_lecture_video_map_df = coders_annotations_df.copy()
coders_lecture_video_map_df["LectureId"] = coders_lecture_video_map_df["Partial LECTURE'S ID"]+coders_lecture_video_map_df["Lecture no."]
coders_lecture_video_map_df.head(1)

,Semester (year when the lecture toook place + semester no.),Semester annonymisation,Title,Start Date,Start Time,End Time,Studio ID (lecture theatre),Author ID,School,Blackboard Course ID,...,Course ID annonymisation,Status,Lecture duration,Approximate lecture duration,INSTRUCTOR'S ID,Partial LECTURE'S ID,Lecture no.,Partial lecture ID without RA no.,Old Obs. Name,LectureId
0,17S1,J3S1,17S1-CM1021-LEC_20170823,23.08.2017,12:30:00,13:26:00,LT23-PC1,sumod,SPMS,17S1-CM1021-LEC,...,C1,Published,00:56:00,1,I12,J3S1C1,L1,J3S1C1L1-I12,a3,J3S1C1L1


### Add video name to coders

In [14]:
coders_lecture_video_map_df = coders_lecture_video_map_df.rename(columns={'Title': 'Video'})
coders_lecture_video_map_df.head(1)

,Semester (year when the lecture toook place + semester no.),Semester annonymisation,Video,Start Date,Start Time,End Time,Studio ID (lecture theatre),Author ID,School,Blackboard Course ID,...,Course ID annonymisation,Status,Lecture duration,Approximate lecture duration,INSTRUCTOR'S ID,Partial LECTURE'S ID,Lecture no.,Partial lecture ID without RA no.,Old Obs. Name,LectureId
0,17S1,J3S1,17S1-CM1021-LEC_20170823,23.08.2017,12:30:00,13:26:00,LT23-PC1,sumod,SPMS,17S1-CM1021-LEC,...,C1,Published,00:56:00,1,I12,J3S1C1,L1,J3S1C1L1-I12,a3,J3S1C1L1


### Leave only coders lecture ID and video name

In [15]:
coders_lecture_video_map_df = coders_lecture_video_map_df[["LectureId", "Video"]]
coders_lecture_video_map_df.head(2)

,LectureId,Video
0,J3S1C1L1,17S1-CM1021-LEC_20170823
1,J3S1C1L2,17S1-CM1021-LEC_20170906


## Check if coders video exists

In [16]:
set_of_coders_videos = coders_lecture_video_map_df["Video"].unique()
coders_video_exists_dict = {}
for video in set_of_coders_videos:
    video_path = os.path.join(C.ORIGINAL_VIDEOS_DIR, str(video) + ".mp4")
    video_exists = os.path.exists(video_path)
    coders_video_exists_dict[video] = video_exists
coders_video_exists_dict

{'17S1-CM1021-LEC_20170823': True,
 '17S1-CM1021-LEC_20170906': True,
 '15S1-MH1800-LEC_20150817': True,
 '15S1-MH1800-LEC_20150824': True,
 '15S1-MH1800-LEC_20150831': True,
 '15S2-MH1402-LEC_20160115': True,
 '15S2-MH1402-LEC_20160122': True,
 '15S2-MH1402-LEC_20160129': True,
 '15S2-MH1402-LEC_20160205': True,
 '15S2-MH1402-LEC_20160212': True,
 '16S1-MH1812-LEC_20160808': True,
 '16S1-MH1812-LEC_20160811': True,
 '16S1-MH1812-LEC_20160815': True,
 '16S1-MH1812-LEC_20160818': True,
 '16S1-MH1812-LEC_20160822': True,
 '16S2-MH1301-LEC_20170109': True,
 '16S2-MH1301-LEC_20170116': True,
 '16S2-MH1301-LEC_20170123': True,
 '16S2-MH1301-LEC_20170206': True,
 '16S2-MH1301-LEC_20170213': True,
 '17S1-MH1100-LEC_20170817': True,
 '17S1-MH1100-LEC_20170818': True,
 '17S1-MH1100-LEC_20170824': True,
 '17S1-MH1100-LEC_20170825': True,
 '17S1-MH1100-LEC_20170831': True,
 '17S1-MH1100-LEC_20171012': True,
 '17S1-MH1100-LEC_20171019': True,
 '17S1-MH1100-LEC_20171020': True,
 '17S1-MH1100-LEC_20

**Result**: still not all videos exists but definately a lot more than in original annotations

## Map lecture id to existing video file

In [17]:
# example lectureId - video mapping
t = coders_lecture_video_map_df[coders_lecture_video_map_df["LectureId"]=="J3S1C1L1"]
t.head()

,LectureId,Video
0,J3S1C1L1,17S1-CM1021-LEC_20170823


In [18]:
# get all lectures
set_of_all_lecture_ids = annotations_df[C.A_LECTURE_ID].unique()
set_of_all_lecture_ids

array(['J1S1C11L1', 'J1S1C11L2', 'J1S1C11L3', 'J1S1C11L4', 'J1S1C12L1',
       'J1S1C13L1', 'J1S1C13L2', 'J1S1C13L3', 'J1S1C13L4', 'J1S1C13L5',
       'J1S1C14L1', 'J1S1C14L10', 'J1S1C14L2', 'J1S1C14L3', 'J1S1C14L4',
       'J1S1C14L5', 'J1S1C14L6', 'J1S1C14L7', 'J1S1C14L8', 'J1S1C14L9',
       'J1S1C5L1', 'J1S1C5L2', 'J1S1C5L3', 'J1S2C15L1', 'J1S2C15L2',
       'J1S2C15L3', 'J1S2C15L4', 'J1S2C15L5', 'J1S2C4L1', 'J1S2C4L2',
       'J1S2C4L3', 'J1S2C4L4', 'J1S2C4L5', 'J2S1C10L1', 'J2S1C10L2',
       'J2S1C10L3', 'J2S1C10L4', 'J2S1C10L5', 'J2S1C13L1', 'J2S1C13L2',
       'J2S1C13L3', 'J2S1C13L4', 'J2S1C13L5', 'J2S1C13L6', 'J2S1C17L1',
       'J2S1C17L2', 'J2S1C17L3', 'J2S1C17L4', 'J2S1C17L5', 'J2S2C3L1',
       'J2S2C3L2', 'J2S2C3L3', 'J2S2C3L4', 'J2S2C3L5', 'J3S1C11L1',
       'J3S1C11L2', 'J3S1C11L3', 'J3S1C11L4', 'J3S1C11L5', 'J3S1C17L1',
       'J3S1C17L2', 'J3S1C17L3', 'J3S1C17L4', 'J3S1C17L5', 'J3S1C17L6',
       'J3S1C1L1', 'J3S1C1L2', 'J3S1C2L1', 'J3S1C2L10', 'J3S1C2L2',
       '

In [19]:
lecture_id_to_existsing_video_dict = {}

# for every lecture
for lecture_id in set_of_all_lecture_ids:
    
    # get original annotations for lecture
    annotations_lecture_df = annotations_df[annotations_df[C.A_LECTURE_ID] == lecture_id]
    # also, get all lecture - video rows from coders file
    coders_lecture_df = coders_lecture_video_map_df[coders_lecture_video_map_df["LectureId"] == lecture_id]
    
    # Generally, there should be only one mapping of lecturId - video in both original and coders files.
    # However, this might not be the case, so we assume here that for some reasons (probably errors)
    # some lecture events might be mapped to videoName1, and others to videoName2, videoName3 etc.
    all_original_videos_for_lecture = annotations_lecture_df[C.A_VIDEO].unique()
    all_coders_videos_for_lecture = coders_lecture_df["Video"].unique()
    
    matched_in_coders_file = False
    
    # first try coders file - possibly more up to date
    for coders_video in all_coders_videos_for_lecture:
        video_exists = coders_video_exists_dict[coders_video]
        
        if video_exists:
            lecture_id_to_existsing_video_dict[lecture_id] = coders_video
            matched_in_coders_file = True
            break
            
    if matched_in_coders_file:
        continue
    
    existing_video = None
    
    # if not, try to match video from annotations file
    for video in all_original_videos_for_lecture:
        
        video_exists = video_exists_dict[video]
        
        if video_exists:
            print(f"{lecture_id}: Setting video from annotations file")
            is_different_video_for_same_lecture = video == existing_video
            if is_different_video_for_same_lecture:
                print(f"Duplicate! Lecture: {lecture_id}, Video: {video}")
                raise 1
            
            existing_video = video
    
    lecture_id_to_existsing_video_dict[lecture_id] = existing_video

Lectures without video:

In [20]:
lectures_without_video = [k for (k,v) in lecture_id_to_existsing_video_dict.items() if v is None]
lectures_without_video

['J1S1C14L10', 'J1S1C14L5', 'J1S1C14L6', 'J1S1C14L7', 'J1S1C14L8', 'J1S1C14L9']

### Remove lectures without videos

Those lecture have to be removed - we have no reliable way of finding videos for them

In [21]:
print(len(annotations_df))
annotations_df = annotations_df[~annotations_df[C.A_LECTURE_ID].isin(lectures_without_video)]
print(len(annotations_df))

40946
39753


### Fix video files for lectures

In [22]:
annotations_df[C.A_VIDEO] = annotations_df[C.A_LECTURE_ID].apply(lambda lecture_id: lecture_id_to_existsing_video_dict[lecture_id])

number_of_all_events = annotations_df.shape[0]
number_of_invalid_events = annotations_df[annotations_df[C.A_VIDEO].isnull()].shape[0]

print(f"Number of events to remove: {number_of_invalid_events} / {number_of_all_events} (SHOULD BE ZERO)")

Number of events to remove: 0 / 39753 (SHOULD BE ZERO)


### Show videos (from source directory) without annotations

Now, every annotation can be matched with some video. However, not every video was annotated:

In [23]:
all_video_names = [name for name in os.listdir(C.ORIGINAL_VIDEOS_DIR) if ".mp4" in name]
all_video_names = [name[:-4] for name in all_video_names]

In [24]:
video_to_lecture_id = {v: k for (k,v) in lecture_id_to_existsing_video_dict.items()}

In [25]:
for video in all_video_names:
    if video_to_lecture_id.get(video) is None:
        print(video)

17S2-BS1102-LEC_20180126
16S2-MH1402-LEC_20170324
18S1-BS1101-LEC_20181023
17S1-PH1011-LEC_20170815
17S2-BS1102-LEC_20180125
18S1-BS1101-LEC_20181019
17S2-PH1107-CY1307-LEC_20180213
18S1-BS1101-LEC_20181015
18S1-PH1801-LEC_20180831
15S1-MH1800-LEC_20151012
18S1-BS1101-LEC_20181022
15S1-MH1812-LEC_20150914
PH1011-PHYSICS_20150818
15S1-MH1800-LEC_20150907
17S2-BS1102-LEC_20180123
17S1-CM1021-LEC_20170818
17S1-PH1011-LEC_20170829
18S1-PH1801-LEC_20180907
17S2-CM1041-LEC_20180214
17S2-BS1102-LEC_20180129
17S1-CM1021-LEC_20170825
16S2-MH1402-LEC_20170317
18S1-BS1101-LEC_20181017
17S1-PH1011-LEC_20170905
18S1-PH1801-LEC_20180817
15S1-MH1800-LEC_20151005
17S2-BS1102-LEC_20180122
15S1-MH1800-LEC_20150914
17S1-CM1021-LEC_20170830
17S2-BS1102-LEC_20180202
15S1-MH1812-LEC_20150910
17S2-MH1804-LEC_20180405
16S2-MH1402-LEC_20170310
17S1-PH1011-LEC_20170822
17S2-BS1102-LEC_20180117
17S2-BS1102-LEC_20180130
17S2-CM1041-LEC_20180223
18S1-BS1101-LEC_20181016
BS1101-BASICS OF TCM_20181012
15S1-MH1800-LE

**The amount of not annotated videos is huge**

However, we cannot do anything about it. Moreover, we don't even have to remove them - just leave them alone.

### Remove videos without layout annotations

In [26]:
example_layout_file = os.listdir(C.LAYOUT_ANNOTATIONS_DIR)[0]
example_layout_file

'16S2-MH1402-LEC_20170331.pt'

In [27]:
example_layout_labels = torch.load(f"{C.LAYOUT_ANNOTATIONS_DIR}/{example_layout_file}")
example_layout_labels.shape

torch.Size([6136])

In [28]:
videos_with_layout = [file[:-3] for file in os.listdir(C.LAYOUT_ANNOTATIONS_DIR)]
print(videos_with_layout[0])
print(len(videos_with_layout))

16S2-MH1402-LEC_20170331
179


In [29]:
all_videos = [file[:-3] for file in os.listdir(C.ORIGINAL_VIDEOS_DIR)]
print(all_videos[0])
print(len(all_videos))

17S2-BS1102-LEC_20180126.
188


In [30]:
len(annotations_df)

39753

In [31]:
annotations_df_tmp = annotations_df[C.A_VIDEO].isin(videos_with_layout)
len(annotations_df_tmp)

39753

Seems like all videos were annotated with layout type

### Fix NaN duration

In [38]:
annotations_df = annotations_df.fillna(0)

### Add time column

In [39]:
annotations_df[C.A_TIME] = (annotations_df[C.A_START] + annotations_df[C.A_STOP]) / 2

### Round time columns

In [40]:
annotations_df[C.A_DURATION] = np.round(annotations_df[C.A_DURATION], 4)
annotations_df[C.A_START] = np.round(annotations_df[C.A_START], 4)
annotations_df[C.A_STOP] = np.round(annotations_df[C.A_STOP], 4)
annotations_df[C.A_TIME] = np.round(annotations_df[C.A_TIME], 4)

In [41]:
annotations_df.head(3)

,Id,Unnamed: 0,Behavior,Behavior type,Behavioral category,Comment start,Comment stop,Duration (s),FPS,Media file,...,Start (s),Stop (s),Subject,Total length,lecture_id,research_assistant_id,subject_id,BehaviorId,Video,Time
0,0,0,eye contact,STATE,Encouraging learner engagement,0.0,0,2841.008,29.83,C:/Users/ACER/Downloads/PH1011-PHYSICS_2015092...,...,29.355,2870.363,No focal subject,7197.58,J1S1C11L1,RA13,J1S1C11,B_EYE,PH1011-PHYSICS_20150922,1449.859
1,1,1,Writing on slides,STATE,Providing clear explanation; Arousing interest,0.0,0,2739.504,29.83,C:/Users/ACER/Downloads/PH1011-PHYSICS_2015092...,...,132.351,2871.855,No focal subject,7197.58,J1S1C11L1,RA13,J1S1C11,B_WRITING_SLIDES,PH1011-PHYSICS_20150922,1502.103
2,2,2,use of voice intonation to Emphasise more impo...,STATE,Encouraging learner engagement; Pacing and mai...,0.0,0,4.002,29.83,C:/Users/ACER/Downloads/PH1011-PHYSICS_2015092...,...,350.396,354.398,No focal subject,7197.58,J1S1C11L1,RA13,J1S1C11,B_VOICE,PH1011-PHYSICS_20150922,352.397


# Add human time

In [48]:
def format_seconds(seconds):
    date = datetime.fromtimestamp(seconds) - timedelta(hours=1)
    return date.strftime("%H:%M:%S:%f")[:-3]

In [50]:
format_seconds(29.355)

'00:00:29:355'

In [54]:
annotations_df[C.A_START_H] = annotations_df.apply(lambda row: format_seconds(row[C.A_START]), axis=1)
annotations_df[C.A_STOP_H] = annotations_df.apply(lambda row: format_seconds(row[C.A_STOP]), axis=1)
annotations_df[C.A_TIME_H] = annotations_df.apply(lambda row: format_seconds(row[C.A_TIME]), axis=1)

In [55]:
annotations_df.head(3)

,Id,Unnamed: 0,Behavior,Behavior type,Behavioral category,Comment start,Comment stop,Duration (s),FPS,Media file,...,Total length,lecture_id,research_assistant_id,subject_id,BehaviorId,Video,Time,Start H (s),Stop H (s),Time H
0,0,0,eye contact,STATE,Encouraging learner engagement,0.0,0,2841.008,29.83,C:/Users/ACER/Downloads/PH1011-PHYSICS_2015092...,...,7197.58,J1S1C11L1,RA13,J1S1C11,B_EYE,PH1011-PHYSICS_20150922,1449.859,00:00:29:355,00:47:50:363,00:24:09:859
1,1,1,Writing on slides,STATE,Providing clear explanation; Arousing interest,0.0,0,2739.504,29.83,C:/Users/ACER/Downloads/PH1011-PHYSICS_2015092...,...,7197.58,J1S1C11L1,RA13,J1S1C11,B_WRITING_SLIDES,PH1011-PHYSICS_20150922,1502.103,00:02:12:351,00:47:51:855,00:25:02:103
2,2,2,use of voice intonation to Emphasise more impo...,STATE,Encouraging learner engagement; Pacing and mai...,0.0,0,4.002,29.83,C:/Users/ACER/Downloads/PH1011-PHYSICS_2015092...,...,7197.58,J1S1C11L1,RA13,J1S1C11,B_VOICE,PH1011-PHYSICS_20150922,352.397,00:05:50:396,00:05:54:398,00:05:52:397


### Save processed annotations

In [56]:
annotations_df.to_csv(C.PROCESSED_ANNOTATIONS_PATH, index=False)